In [13]:
import os
import warnings
from dotenv import load_dotenv, find_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool, \
                         BaseTool

warnings.filterwarnings('ignore')

load_dotenv(find_dotenv())


True

In [14]:
openai_api_key = os.getenv("OPENAPI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

In [15]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify potential high-value leads that fit to the company interests.",
    backstory=(
        "As a part of the dynamic sales team at the company, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True,
)

In [16]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of the company's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

In [17]:
directory_tool = DirectoryReadTool(directory='./lab_assets/customer_oureach_campaing')
read_tool = FileReadTool()
search_tool = SerperDevTool()

In [18]:
class SentimentAnalysisTool(BaseTool):
    name: str = "Sentiment Analysis Tool"
    description: str = ("Analyze the sentiment of a given text "
                        "to ensure positive communication with leads.")
    def _run(self, text: str) -> str:
        # Implementation goes here
        return "positive"

In [19]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [20]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_tool, read_tool, search_tool],
    agent=sales_rep_agent,
)

In [21]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [22]:
crew = Crew(agents=[sales_rep_agent, lead_sales_rep_agent], 
            tasks=[lead_profiling_task, personalized_outreach_task], 
            memory=True,
            verbose=True)

2024-08-13 17:28:42,095 - 11004 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [23]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [2024-08-13 17:28:43][DEBUG]: == Working Agent: Sales Representative
 [2024-08-13 17:28:43][INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: To compile a detailed profile on DeepLearningAI, I'll first look for general information about the company through an internet search. This will help gather initial data about their background, key personnel, and recent business developments.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company profile"} 


Search results: T

In [25]:
from IPython.display import Markdown
Markdown(result.raw)

Subject: Empowering AI Education Together

Dear Andrew Ng,

I hope this message finds you well. I'm reaching out to express our admiration for DeepLearningAI's recent initiatives to expand AI education globally. Your efforts in launching new courses like 'Building Systems with the ChatGPT API' and collaborations with major tech entities are truly inspiring.

At [Your Company], we specialize in enhancing online educational experiences through advanced analytics and interactive AI-driven platforms. Given DeepLearningAI's commitment to practical AI applications and global education, we see a significant alignment in our goals.

We believe our technologies could further empower DeepLearningAI by providing detailed insights into student engagements and optimizing learning outcomes. Imagine a world where every student's needs are predicted and met with precision, enhancing their learning journey.

Could we schedule a call to discuss potential collaborations? I am confident that together, we can achieve remarkable strides in making AI education even more accessible and impactful.

Looking forward to your thoughts.

Warm regards,

[Your Name]
Lead Sales Representative
[Your Company]